# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

> [!NOTE]  
> Because of computing resources issues, the code contained here uses a sample of 60 items of the entire dataset 

In [2]:
%load_ext dotenv
%dotenv
import sys
import os
sys.path.append("../../05_src")
sys.path.append(os.getenv('PRICE_DATA'))

In [3]:
import dask.dataframe as dd

c:\Users\sagra\.conda\envs\dsi_participant\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [23]:
import os
from glob import glob

import dask.dataframe as dd
import pandas as pd
from utils.logger import get_logger
_logs = get_logger(__name__)

PRICE_DATA = os.getenv('PRICE_DATA')

price_paths = glob(os.path.join(PRICE_DATA, "**/**/*.parquet"), recursive = True)
print(len(price_paths))

import random
random.seed(42)
price_paths = random.sample(price_paths, 2_000)

8880


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [24]:
# Read all files as one dask DataFrame
dd_px = dd.read_parquet(price_paths).set_index("ticker")

dd_group = dd_px.groupby('ticker', group_keys=False)

# Close
dd_shift = dd_group.apply(
    lambda x: x.assign(close_lag_1 = x['Close'].shift(1))
    )

dd_shift = dd_shift.assign(
    adj_close_lag_1 = lambda x: x['Adj Close'].shift(1)
    )

dd_rets = dd_shift.assign(
    returns = lambda x: x['Close']/x['close_lag_1'] - 1
    )

dd_rets = dd_rets.assign(
    adj_returns = lambda x: x['Adj Close']/x['adj_close_lag_1'] - 1
    )

dd_feat = dd_rets.assign(
    hi_lo_range = lambda x: x['Close'].max() - x['Close'].min()
)

dd_feat = dd_feat.assign(
    adj_hi_lo_range = lambda x: x['Adj Close'].max() - x['Adj Close'].min()
)


C:\Users\sagra\AppData\Local\Temp\ipykernel_16988\2662748605.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_group.apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# SKIP THIS CELL
# CODE CORRESPONDING TO THE ABOVE BULLET-POINT LIST IS GIVEN IN THE NEXT CELL

import time


# New feature on Dask (before converting to pd.DataFrame)
start_0 = time.time()
dd_feat.assign(
    avg_10 = lambda x: x['returns'].rolling(10).mean()
    )
end_0 = time.time()

_logs.info('Dask DataFrame New feature: {:.3e}'.format(end_0-start_0))

2025-06-08 23:10:55,880, 72333521.py, 12, INFO, Dask DataFrame New feature: 1.380e-02


In [26]:
# Convert to pd.DataFrame
start_1 = time.time()
pd_feat = dd_feat.compute()
end_1 = time.time()

# New feature
start_2 = time.time()

pd_feat = pd_feat.assign(
    avg_10 = lambda x: x['returns'].rolling(10).mean()
    )

end_2 = time.time()

pd_feat = pd_feat.assign(adj_avg_10 = lambda x: x['adj_returns'].rolling(10).mean())


display(pd_feat.head(3))
_logs.info('Dask or pd.DataFrame after rolling: {}'.format(type(pd_feat)))
_logs.info('Conversion time: {:.2f} New feature: {:.3e}'.format(end_1-start_1, end_2-start_2))

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,close_lag_1,adj_close_lag_1,returns,adj_returns,hi_lo_range,adj_hi_lo_range,avg_10,adj_avg_10
ticker,,,,,,,,,,,,,,,,,
ACN,2002-07-03,16.889999,17.750000,16.040001,17.50,13.156021,3582400.0,ACN.csv,2002,NaN,NaN,NaN,NaN,1966.7754,1966.784554,NaN,NaN
ACN,2002-07-05,17.799999,18.299999,17.719999,17.90,13.456730,1190200.0,ACN.csv,2002,17.5,13.156021,0.022857,0.022857,1966.7754,1966.784554,NaN,NaN
ACN,2002-07-08,17.700001,17.750000,17.150000,17.57,13.208645,5472700.0,ACN.csv,2002,17.9,13.456730,-0.018436,-0.018436,1966.7754,1966.784554,NaN,NaN


2025-06-08 23:13:19,386, 2796368945.py, 19, INFO, Dask or pd.DataFrame after rolling: <class 'pandas.core.frame.DataFrame'>
2025-06-08 23:13:19,387, 2796368945.py, 20, INFO, Conversion time: 13.59 New feature: 3.091e-02


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

> - [1] No. The moving average return could have been obtained without transforming the data.
>
> - [2] Indeed, the transformation and subsequent feature obtention has added computing cost to the entire process:
>   - Benefits from Dask distributed computing are not being used anymore
>   - The computing time of obtaining the rolling feature by using a pd.DataFrame is larger than if dask had been used

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.